In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
import os
from PIL import Image


# Get the absolute path to the dataset
dataset_path = r'C:\Users\Basuru Yasaruwan\Contacts\Desktop\IPCV Practicals\Practical 03\IM02\dataset'
absolute_path = os.path.abspath(dataset_path)

# Load the pre-trained VGG16 model
base_model = tf.keras.applications.VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
base_model.trainable = False

# Create a custom model for fine-tuning
model = tf.keras.Sequential([
    base_model,
    # reduce the complexity of the prev tensor derived from the input
    # by averaging the pixel distributions. So learning will be easy
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(256, activation='relu'),
    # randomly deactivate nerouns to prevent overfitting on features by 
    # some neurons. 0.5 represent the % of neurons impacted by the dropout 
    #feature
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(1, activation='sigmoid')  # Binary classification (gun or not)
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Data preparation with data augmentation. Larger batch sizes are good to
# make the models to be more mature. However, it could lead to stuck due to
# memory shortages. Hence, trial and error expertimentations are required.
batch_size = 32
# augment the images on the go as per the specs given during the training
train_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)
# configured pipeline of datagenerator is directly linked to the folder
train_generator = train_datagen.flow_from_directory(
    absolute_path,
    target_size=(224, 224),
    batch_size=batch_size,
    # augment the images considering correct and wrong representations.
    # this allow the model to learn with a comparative assesment.
    class_mode='binary',
    # can be either training or validation. In validation mode, adjust the
    # images to support the inferencing purposes.
    subset='training'
)

# arrangement of the validation dataset.
validation_generator = train_datagen.flow_from_directory(
    absolute_path,
    target_size=(224, 224),
    batch_size=batch_size,
    class_mode='binary',
    subset='validation'
)

# Train the model with augmented data
model.fit(
    train_generator,
    epochs=50,
    validation_data=validation_generator
)

# Save the model
model.save('custom_gun_model.h5')






Found 32 images belonging to 2 classes.
Found 6 images belonging to 2 classes.
Epoch 1/50


1/1 [==============================] - 4s 4s/step - loss: 0.7982 - accuracy: 0.3438 - val_loss: 0.6993 - val_accuracy: 0.5000
Epoch 2/50
1/1 [==============================] - 3s 3s/step - loss: 0.6625 - accuracy: 0.5312 - val_loss: 0.7038 - val_accuracy: 0.5000
Epoch 3/50
1/1 [==============================] - 3s 3s/step - loss: 0.7242 - accuracy: 0.4375 - val_loss: 0.7176 - val_accuracy: 0.5000
Epoch 4/50
1/1 [==============================] - 3s 3s/step - loss: 0.6247 - accuracy: 0.5938 - val_loss: 0.7313 - val_accuracy: 0.5000
Epoch 5/50
1/1 [==============================] - 3s 3s/step - loss: 0.7119 - accuracy: 0.5312 - val_loss: 0.6939 - val_accuracy: 0.5000
Epoch 6/50
1/1 [==============================] - 3s 3s/step - loss: 0.6721 - accuracy: 0.5938 - val_loss: 0.6778 - val_accuracy: 0.6667
Epoch 7/50
1/1 [==============================] - 3s 3s/step - loss: 0.7840 - accuracy: 0.375

C:\Users\Basuru Yasaruwan\Contacts\Desktop\IPCV Practicals\Practical 03\IM02\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [2]:
# Test the prediction capability with a new image
new_image_path = r'C:\Users\Basuru Yasaruwan\Contacts\Desktop\IPCV Practicals\Practical 03\IM02\dataset\train\no_gun\1.jpg'  
# Replace with the path to your new image

# Load the new image. It also need to be pre-process as to suite with the
#configured model. Becz model has been trained with configured images and
# we need the same during the inferencing as well.

new_image = load_img(new_image_path, target_size=(224, 224))
new_image_array = img_to_array(new_image)
# format the image data as needed by a deep learning model
new_image_array = tf.expand_dims(new_image_array, 0)  # Add batch dimension
new_image_array /= 255.0  # Rescale pixel values to [0, 1]

# Make predictions
prediction = model.predict(new_image_array)

# Display the prediction
if prediction > 0.5:
    print("Prediction: Gun Detected")
else:
    print("Prediction: No Gun Detected")


1/1 [==============================] - 0s 249ms/step
Prediction: No Gun Detected
